In [1]:
!pip install matplotlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.datasets import cifar100
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense, UpSampling2D, Dropout, BatchNormalization, RandomFlip, RandomTranslation, RandomRotation,RandomBrightness, RandomContrast, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint
from tensorflow.keras.applications.resnet_v2 import preprocess_input
import re

2024-04-09 10:18:23.522338: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-09 10:18:23.603570: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 10:18:24.750283: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
n_epoch = 100
batch_size = 100
taux_validation = 0.1
num_classes = 100
n_images = 50000 # Pour l'entrainement, et 10000 pour le test

In [3]:
def preprocessing(image, label):
    image = tf.image.resize(image, (224, 224))
    label = tf.squeeze(tf.one_hot(label, depth = num_classes), axis = 0)
    return  image, label

augmentation_donnees_keras = Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2),
    RandomBrightness(0.2,value_range=(0,1))
])

def augmentation_donnees(image, label):
    return augmentation_donnees_keras(image/255.0, training = True)*255.0, label

def preprocess_resnet(image, label):
    return preprocess_input(image), label

train_dataset, test_dataset = cifar100.load_data()

validation_size = int(n_images * taux_validation)
train_size = n_images - validation_size
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset).map(preprocessing).shuffle(n_images, reshuffle_each_iteration=False)
test_dataset = tf.data.Dataset.from_tensor_slices(test_dataset).map(preprocessing)

validation_dataset = train_dataset.take(validation_size).batch(batch_size).map(preprocess_resnet).prefetch(tf.data.AUTOTUNE)
train_dataset = train_dataset.skip(validation_size).batch(batch_size).map(augmentation_donnees).map(preprocess_resnet).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).map(preprocess_resnet).prefetch(tf.data.AUTOTUNE)

2024-04-09 10:18:25.773030: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12035 MB memory:  -> device: 0, name: NVIDIA A2, pci bus id: 0000:b1:00.0, compute capability: 8.6


In [4]:
model = Sequential([
    ResNet50V2(include_top=False, weights='imagenet', pooling="avg"),
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
])

In [ ]:
!mkdir -p sauvegardes
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, start_from_epoch=20)
csv_logger = CSVLogger("sauvegardes/modele_enseignant_logs.csv", separator=",", append=False)
model_checkpoint_callback = ModelCheckpoint(filepath="sauvegardes/modele_enseignant_checkpoint.keras", monitor='val_loss', save_best_only=True)

for layer in model.layers:
    if layer.name == "resnet50v2":
        layer.trainable = False
    break
model.fit(train_dataset, epochs=n_epoch, batch_size=batch_size, validation_data = validation_dataset, callbacks = [early_stopping_callback, csv_logger, model_checkpoint_callback])

Epoch 1/100


I0000 00:00:1712657924.647245   94486 service.cc:145] XLA service 0x7f6978001eb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1712657924.647294   94486 service.cc:153]   StreamExecutor device (0): NVIDIA A2, Compute Capability 8.6
2024-04-09 10:18:44.906589: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-09 10:18:46.143498: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900
I0000 00:00:1712657929.290208   94675 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_6102', 108 bytes spill stores, 108 bytes spill loads

I0000 00:00:1712657929.529313   94702 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8613', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1712657929.8

450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.0883 - loss: 8.0063

I0000 00:00:1712658836.335290   95093 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1542', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1712658836.779446   95117 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1542', 244 bytes spill stores, 244 bytes spill loads

I0000 00:00:1712658837.045361   95096 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1542', 300 bytes spill stores, 184 bytes spill loads



450/450 ━━━━━━━━━━━━━━━━━━━━ 939s 2s/step - accuracy: 0.0883 - loss: 8.0013 - val_accuracy: 0.2880 - val_loss: 4.2230
Epoch 2/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 944s 2s/step - accuracy: 0.1506 - loss: 4.5361 - val_accuracy: 0.3142 - val_loss: 3.9845
Epoch 3/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 933s 2s/step - accuracy: 0.1752 - loss: 4.4037 - val_accuracy: 0.3348 - val_loss: 3.8961
Epoch 4/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 948s 2s/step - accuracy: 0.1865 - loss: 4.3473 - val_accuracy: 0.3444 - val_loss: 3.8351
Epoch 5/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1955 - loss: 4.3229

/opt/mamba/lib/python3.11/site-packages/keras/src/callbacks/model_checkpoint.py:199: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=self._current_epoch, batch=batch, logs=logs)


450/450 ━━━━━━━━━━━━━━━━━━━━ 963s 2s/step - accuracy: 0.1955 - loss: 4.3229 - val_accuracy: 0.3472 - val_loss: 3.8080
Epoch 6/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 942s 2s/step - accuracy: 0.1994 - loss: 4.3040 - val_accuracy: 0.3594 - val_loss: 3.7823
Epoch 7/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 932s 2s/step - accuracy: 0.2037 - loss: 4.2827 - val_accuracy: 0.3706 - val_loss: 3.7573
Epoch 8/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 907s 2s/step - accuracy: 0.2091 - loss: 4.2677 - val_accuracy: 0.3742 - val_loss: 3.7444
Epoch 9/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 900s 2s/step - accuracy: 0.2133 - loss: 4.2634 - val_accuracy: 0.3694 - val_loss: 3.7375
Epoch 10/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 825s 2s/step - accuracy: 0.2175 - loss: 4.2552 - val_accuracy: 0.3858 - val_loss: 3.7088
Epoch 11/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 891s 2s/step - accuracy: 0.2173 - loss: 4.2393 - val_accuracy: 0.3718 - val_loss: 3.7147
Epoch 12/100
450/450 ━━━━━━━━━━━━━━━━━━━━ 820s 2s/step - accuracy: 0.2174 - loss: 4.2471 - val_accur

In [ ]:
model.save("sauvegardes/modele_enseignant_transfert.keras")

In [ ]:
model.evaluate(test_dataset)

In [ ]:
model.trainable = True
model.compile(optimizer=keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
csv_logger = CSVLogger("sauvegardes/modele_enseignant_logs.csv", separator=",", append=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint_callback = ModelCheckpoint(filepath="sauvegardes/modele_enseignant_checkpoint.keras", monitor='val_loss', save_best_only=True)
model.fit(train_dataset, epochs=40, batch_size=batch_size, validation_data = validation_dataset, callbacks = [early_stopping_callback, csv_logger, model_checkpoint_callback])

In [ ]:
model.save("sauvegardes/modele_enseignant.keras")

In [ ]:
model.evaluate(test_dataset)

In [ ]:
!mc cp /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras s3/afeldmann/modele_enseignant.keras